## Risorse
- https://www.kaggle.com/code/vitalykudelya/explain-dataset-and-baseline
- 

## Imports

In [ ]:
import pathlib

import pandas as pd
import plotly.express as px

## Environment variables

In [ ]:
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = '{:.3f}'.format

## Constants definition

In [ ]:
TRAIN_PATH: pathlib.Path = pathlib.Path().absolute().parent.parent / "data" / "predict-energy-behavior-of-prosumers" / "train.csv"
TRAIN_PATH

## Data load

- **origin_date** - The date when the day-ahead prices became available.
- **forecast_date** - Represents the start of the 1-hour period when the price is valid
- **euros_per_mwh** - The price of electricity on the day ahead markets in euros per megawatt hour.
- **data_block_id** - All rows sharing the same `data_block_id` will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather `data_block_id` for predictins made on October 31st is 100 then the historic weather `data_block_id` for October 31st will be 101 as the historic weather data is only actually available the next day.

In [32]:
train: pd.DataFrame = pd.read_csv(TRAIN_PATH)
train.query("data_block_id == 637").sort_values(by=["prediction_unit_id", "datetime"]).reset_index(drop=True)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,2.403,0,2023-05-31 00:00:00,637,2015232,0
1,0,0,1,516.958,1,2023-05-31 00:00:00,637,2015233,0
2,0,0,1,2.336,0,2023-05-31 01:00:00,637,2015362,0
3,0,0,1,470.284,1,2023-05-31 01:00:00,637,2015363,0
4,0,0,1,1.810,0,2023-05-31 02:00:00,637,2015492,0
...,...,...,...,...,...,...,...,...,...
3115,14,1,2,6.911,1,2023-05-31 21:00:00,637,2018079,68
3116,14,1,2,0.017,0,2023-05-31 22:00:00,637,2018208,68
3117,14,1,2,10.401,1,2023-05-31 22:00:00,637,2018209,68
3118,14,1,2,0.000,0,2023-05-31 23:00:00,637,2018338,68


In [27]:
train.columns

Index(['county', 'is_business', 'product_type', 'target', 'is_consumption',
       'datetime', 'data_block_id', 'row_id', 'prediction_unit_id'],
      dtype='object')

In [ ]:
train.dtypes

## EDA

### Check database info

In [ ]:
train.info(verbose=True, memory_usage="deep", show_counts=True)

In [ ]:
train.describe()

## Primary key investigation

In [ ]:
train.groupby(["county", "product_type", "is_business", "is_consumption", "datetime"]).agg({"target": "count"})

In [ ]:
px.line(train, x="datetime", y="target", color="prediction_unit_id")

In [24]:
train["datetime"] = pd.to_datetime(train["datetime"], format="%Y-%m-%d %H:%M:%S")
train["hour"] = train["datetime"].dt.hour

## Conclusion

1. molto semplice, bisogna solo convertire le colonne